In [73]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import math
import torch
import torchvision
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sn
import matplotlib.pyplot as plt

In [74]:
# http://archive.ics.uci.edu/ml/datasets/connectionist+bench+(sonar,+mines+vs.+rocks)
# Connectionist bench sonar, mines vs rocks dataset
df = pd.read_csv('sonar_dataset.csv', header=None)

In [75]:
x = df.drop(60, axis='columns')
y = df[60]

In [76]:
y = pd.get_dummies(y, drop_first=True)

In [77]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=1)

In [78]:
class TrainDataset(Dataset):
    def __init__(self):
        self.x = torch.tensor(x_train.values, dtype=torch.float32)
        self.y = torch.tensor(y_train.values, dtype=torch.float32)
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [79]:
traindataset = TrainDataset()
dataloader_train = DataLoader(traindataset,
                       batch_size=8,
                       shuffle=True,
                       num_workers=0,
                       drop_last=False,
                       pin_memory=True)

In [80]:
class Rock_Classifier(torch.nn.Module):
    def __init__(self):
        super(Rock_Classifier, self).__init__()
        self.layer1 = torch.nn.Linear(60, 30)
        self.layer2 = torch.nn.Linear(30, 15)
        self.layer3 = torch.nn.Linear(15, 1)
        self.sigmoid = torch.nn.Sigmoid()
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        out = self.relu(self.layer1(x))
        out = self.relu(self.layer2(out))
        out = self.sigmoid(self.layer3(out))
        return out

In [81]:
rock_classifier = Rock_Classifier()

In [82]:
optimizer = torch.optim.Adam(rock_classifier.parameters(), lr=0.008, betas=(0.9, 0.99))

In [83]:
loss_function = torch.nn.BCELoss()

In [84]:
for epoch in range(200):
    total_loss = 0.0
    for idx, (x1, y1) in enumerate(dataloader_train):
        optimizer.zero_grad()
        pred = rock_classifier(x1)
        lossval = loss_function(pred, y1)
        lossval.backward(retain_graph=True)
        optimizer.step()
        
        total_loss += lossval.item()
    average_loss = total_loss / traindataset.__len__()
    if epoch % 5 == 0:
        print("Average loss: {}".format(average_loss))

Average loss: 0.089310938731218
Average loss: 0.05475617200136185
Average loss: 0.054072072013066366
Average loss: 0.043147880249680616
Average loss: 0.041364311742094845
Average loss: 0.033809553067653604
Average loss: 0.02624896107814633
Average loss: 0.027095768672342483
Average loss: 0.022612753467491038
Average loss: 0.03879815836150486
Average loss: 0.01899084822537425
Average loss: 0.011296203185338527
Average loss: 0.03382083300190667
Average loss: 0.026179155728851374
Average loss: 0.006058056185466763
Average loss: 0.008043689120453425
Average loss: 0.004642979006558352
Average loss: 0.002335999028545279
Average loss: 0.019397578273081962
Average loss: 0.0021140544567853606
Average loss: 0.0014364713154300728
Average loss: 0.0012370735930744559
Average loss: 0.001209974577180289
Average loss: 0.043339985709444126
Average loss: 0.0021441651210923926
Average loss: 0.0008903969375797971
Average loss: 0.0007545767279318683
Average loss: 0.0009176396958700467
Average loss: 0.00550

In [85]:
class TestDataset(Dataset):
    def __init__(self):
        self.x = torch.tensor(x_test.values, dtype=torch.float32)
        self.y = torch.tensor(y_test.values, dtype=torch.float32)
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [86]:
testdataset = TestDataset()
dataloader_test = DataLoader(testdataset,
                       batch_size=1,
                       shuffle=True,
                       num_workers=0,
                       drop_last=False,
                       pin_memory=True)

In [87]:
rock_classifier = rock_classifier.cpu()
rock_classifier.eval()

y_test_true = []
y_test_pred = []
for idx, (x1, y1) in enumerate(dataloader_test):
    y_test_true.append(y1.numpy()[0][0])
    pred = rock_classifier(x1)
    pred_val = np.round(pred.detach().numpy()[0][0])
    y_test_pred.append(pred_val)

In [88]:
dataloader_train_2 = DataLoader(traindataset, batch_size=1, shuffle=True, num_workers=0, drop_last=False, pin_memory=True)

y_train_true = []
y_train_pred = []
for idx, (x1, y1) in enumerate(dataloader_train_2):
    y_train_true.append(y1.numpy()[0][0])
    pred = rock_classifier(x1)
    pred_val = np.round(pred.detach().numpy()[0][0])
    y_train_pred.append(pred_val)

In [89]:
print(classification_report(y_train_true, y_train_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        84
         1.0       1.00      1.00      1.00        72

    accuracy                           1.00       156
   macro avg       1.00      1.00      1.00       156
weighted avg       1.00      1.00      1.00       156



In [90]:
print(classification_report(y_test_true, y_test_pred))

              precision    recall  f1-score   support

         0.0       0.71      0.89      0.79        27
         1.0       0.83      0.60      0.70        25

    accuracy                           0.75        52
   macro avg       0.77      0.74      0.74        52
weighted avg       0.77      0.75      0.74        52

